In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, MetaData, text
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
load_dotenv()
DB_URL = os.getenv("DB_VM_URL")
engine = create_engine(DB_URL)
conn = engine.connect()
metadata = MetaData()
metadata.reflect(bind=engine)

In [3]:
# select 26 rows from SQL table to insert in dataframe.
query = text('''
            select c.ContactPersoonId, i.CampagneId, i.CampagneNaam, ca.Startdatum, a.Ondernemingsaard, a.Ondernemingstype, a.PrimaireActiviteit, f.Naam as Functie
            from Contactfiche c
            join Account a on a.AccountId = c.AccountId
            join Inschrijving i on i.ContactficheId = c.ContactPersoonId
            join Campagne ca on ca.CampagneId = i.CampagneId
            join ContactficheFunctie cf on cf.ContactpersoonId = c.ContactPersoonId
            join Functie f on f.FunctieId = cf.FunctieId
            where i.CampagneId is not null and a.status = 'actief';
            ''')
result = conn.execute(query)

#convert to dataframe
df_omschrijving = pd.DataFrame(result.fetchall())
df_omschrijving.set_index('ContactPersoonId', inplace=True)
df_omschrijving["rating"] = 5

query2 = text('''
                with pageview_count as(
                    select p.PageTitle, count(p.ContactId) as count
                    from Pageviews p
                    group by p.PageTitle
                )
                select PageTitle, ContactId
                from Pageviews
                where Pagetitle in (select top (2500) PageTitle
                from pageview_count
                order by count desc)
              ''')

result2 = conn.execute(query2)
df_pageviews = pd.DataFrame(result2.fetchall())
df_pageviews.set_index('ContactId', inplace=True)
df_pageviews["rating"] = 1


df_omschrijving.head()

,CampagneId,CampagneNaam,Startdatum,Ondernemingsaard,Ondernemingstype,PrimaireActiviteit,Functie,rating
ContactPersoonId,,,,,,,,
9E22A289-C6F3-ED11-8849-6045BD895CDC,044F17B7-A8B6-ED11-83FF-6045BD895CDC,OV-JO-Stamgasten 2023,2023-05-23 19:00:00,Diensten,Bedrijf,Financiële diensten,Verantwoordelijke Financieel,5
12C6CFE5-5020-EC11-8124-001DD8B72B61,0A14941A-711C-EC11-8123-001DD8B72B61,OV-Community Bouw en Vastgoed 2022,2022-01-31 08:00:00,Diensten,Familiebedrijf,Bouw,Verantwoordelijke Commercieel,5
EC6D5B41-BA4A-E711-80E7-001DD8B72B61,A534E7FC-EEF3-E711-80EE-001DD8B72B62,OV-NW-Voka Update: The Big Refresh 4,2018-02-26 06:00:00,Diensten,Bedrijf,Grafische industrie en diensten,Bedrijfsleider,5
6CE844CB-18AD-ED11-AAD0-6045BD895BFB,31E7F5EB-15FA-ED11-8849-6045BD8952D3,OV-Effectief leiderschap-nj 2023,2023-11-21 09:00:00,Diensten,Bedrijf,Overige industrie & diensten,Verantwoordelijke Commercieel,5
01218324-0942-E611-80D6-005056B06EC4,AF18B713-6DB5-E711-80EC-001DD8B72B62,OV-NW-Voka connect Aalst 2018,2018-01-18 06:00:00,Productie & Diensten,Bedrijf,Technologische industrie & diensten,Bedrijfsleider,5


In [4]:
#remove the rows  [CampagneId, CampagneNaam, Startdatum] from df_omschrijving into a new dataframe
df_campagne = df_omschrijving.iloc[:, 0:3]
df_campagne = df_campagne[df_campagne['Startdatum'] > '2022-11-06']
df_campagne['rating'] = 10
df_omschrijving.drop(df_omschrijving.columns[[0, 1, 2]], axis=1, inplace=True)
df_omschrijving.drop_duplicates(inplace=True)
df_campagne

,CampagneId,CampagneNaam,Startdatum,rating
ContactPersoonId,,,,
9E22A289-C6F3-ED11-8849-6045BD895CDC,044F17B7-A8B6-ED11-83FF-6045BD895CDC,OV-JO-Stamgasten 2023,2023-05-23 19:00:00,10
6CE844CB-18AD-ED11-AAD0-6045BD895BFB,31E7F5EB-15FA-ED11-8849-6045BD8952D3,OV-Effectief leiderschap-nj 2023,2023-11-21 09:00:00,10
01536C51-37E4-ED11-A7C7-6045BD895FE3,29D3979C-A3B9-ED11-83FF-6045BD8956C9,OV-Financieel management voor niet-financiëlen...,2023-06-01 09:00:00,10
85A2204B-733D-EC11-8125-001DD8B72B61,A8949A21-6CEE-ED11-8849-6045BD8956C9,OV-NW&IN-EDIH-DIGITALIS-XpertFinder-Data-AI,2023-10-19 08:00:00,10
F9877768-7826-EC11-8127-001DD8B72B62,E5232B4E-4AE0-ED11-A7C6-6045BD895FE3,OV-IN-Lerend netwerk-Data&AI-2023,2023-11-15 15:00:00,10
...,...,...,...,...
06DB6653-AA1E-ED11-B83D-000D3AAD783A,01FF068A-26D5-ED11-A7C7-6045BD895FE3,OV-NW-Voka|VeGHO Portlunch met Grégoire Dallem...,2023-09-22 14:00:00,10
06DB6653-AA1E-ED11-B83D-000D3AAD783A,A2F63005-8EA8-ED11-AAD1-6045BD895D85,OV-NW-Voka Bilan 2023,2023-05-25 16:00:00,10
06DB6653-AA1E-ED11-B83D-000D3AAD783A,40F621C9-2455-ED11-BBA1-6045BD895BFB,OV-NW-Voka Cluster Sifferdok,2022-12-06 09:30:00,10


In [5]:
df_omschrijving

,Ondernemingsaard,Ondernemingstype,PrimaireActiviteit,Functie,rating
ContactPersoonId,,,,,
9E22A289-C6F3-ED11-8849-6045BD895CDC,Diensten,Bedrijf,Financiële diensten,Verantwoordelijke Financieel,5
12C6CFE5-5020-EC11-8124-001DD8B72B61,Diensten,Familiebedrijf,Bouw,Verantwoordelijke Commercieel,5
EC6D5B41-BA4A-E711-80E7-001DD8B72B61,Diensten,Bedrijf,Grafische industrie en diensten,Bedrijfsleider,5
6CE844CB-18AD-ED11-AAD0-6045BD895BFB,Diensten,Bedrijf,Overige industrie & diensten,Verantwoordelijke Commercieel,5
01218324-0942-E611-80D6-005056B06EC4,Productie & Diensten,Bedrijf,Technologische industrie & diensten,Bedrijfsleider,5
...,...,...,...,...,...
DEE2D349-D179-E311-B4EF-005056B06EC4,Productie,Bedrijf,Energie,Management Assistent,5
D4A84FA2-D9D6-E811-80F8-001DD8B72B62,Productie,Bedrijf,Energie,Medewerker Export,5
A2F1639E-CCED-E611-80E4-001DD8B72B62,Productie,Bedrijf,Energie,Verantwoordelijke Aankoop,5


In [6]:
df_pageviews.head()

,PageTitle,rating
ContactId,,
71C7CDFA-379C-EB11-811E-001DD8B72B62,Weer hogere loonindexering op komst in 2024 | ...,1
2A04879B-1141-EC11-8125-001DD8B72B61,Nieuwjaarsinvitito 2023 | Voka,1
1E9BCBC0-9025-EC11-8124-001DD8B72B61,Netwerkevents | Voka,1
3F1EB182-8E3C-E811-80EF-001DD8B72B62,Marketing | Voka,1
5073196D-6611-E811-80EF-001DD8B72B61,Bedrijfsafval: dit verandert op 1 januari | Voka,1


In [56]:
df_pivot_omschrijving = pd.pivot_table(df_omschrijving, index='ContactPersoonId', columns=['Ondernemingsaard', 'Ondernemingstype', 'PrimaireActiviteit', 'Functie'], values='rating', fill_value = 0).fillna(0).astype('int8')
df_pivot_omschrijving.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2965 entries, 00223C8E-467F-E311-BBFD-005056B06EB4 to FFFAE2B6-11D5-EC11-A7B5-000D3ABD1F85
Columns: 3448 entries, ('Diensten', 'Bedrijf', 'Accountancy & boekhouding', 'Bedrijfsleider') to ('Productie & Diensten', 'Vrije beroepen', 'Farmacie', 'Verantwoordelijke Financieel')
dtypes: int8(3448)
memory usage: 9.8+ MB


In [43]:
df_pivot_campagne = pd.pivot_table(df_campagne, index='ContactPersoonId', columns=['CampagneId'], values='rating', fill_value = 0).fillna(0).astype('int8')
df_pivot_campagne.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5999 entries, 00223C8E-467F-E311-BBFD-005056B06EB4 to FFFEA9CB-ED93-EC11-B400-000D3A2B10EB
Columns: 394 entries, 0056C6DB-6227-ED11-9DB1-002248801038 to FFE56B2D-B848-EE11-BE6E-6045BD895D78
dtypes: int8(394)
memory usage: 2.3+ MB


In [44]:
df_pivot_pageviews = pd.pivot_table(df_pageviews, index='ContactId', columns=['PageTitle'], values='rating', fill_value = 0).fillna(0).astype('int8')
df_pivot_pageviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18505 entries, 00013C8A-6F1C-E211-9DAA-005056B06EB4 to FFD685FA-1F7B-ED11-81AD-6045BD895CDC
Columns: 2499 entries, "Aansluiten bij een lerend netwerk van Voka is de investering meer dan waard!" | Voka to | Voka
dtypes: int8(2499)
memory usage: 44.2+ MB


In [54]:
#join both dataframes
#df_pivot = df_pivot_omschrijving.join(df_pivot_pageviews, how='inner')
df_pivot = pd.concat([df_pivot_omschrijving, df_pivot_pageviews, df_pivot_campagne], axis=1, join='outer').fillna(0).astype('int8')
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25110 entries, 00223C8E-467F-E311-BBFD-005056B06EB4 to FFFEA9CB-ED93-EC11-B400-000D3A2B10EB
Columns: 6341 entries, ('Diensten', 'Bedrijf', 'Accountancy & boekhouding', 'Bedrijfsleider') to FFE56B2D-B848-EE11-BE6E-6045BD895D78
dtypes: int8(6341)
memory usage: 152.0+ MB


In [46]:
similarity_matrix = cosine_similarity(df_pivot, [df_pivot.loc["01B45481-0877-E911-80FE-001DD8B72B62"]]).reshape(1,-1)[0]
similarity_matrix

array([0.        , 0.51256783, 0.        , ..., 0.        , 0.        ,
       0.40522045])

In [47]:
select_contact = "C39094D7-60B1-EB11-811F-001DD8B72B62"

similarities = df_pivot.index.join(similarity_matrix)
similarities = pd.DataFrame({'ContactPersoonId':df_pivot.index, 'sim':similarity_matrix}).set_index('ContactPersoonId')

similar_users = similarities[similarities['sim'] > 0.5].sort_values(by='sim', ascending=False)
similar_users

,sim
ContactPersoonId,
01B45481-0877-E911-80FE-001DD8B72B62,1.000000
B03F4BC2-CF2C-EC11-8124-001DD8B72B61,0.700696
5CF59CF7-7E71-ED11-9561-6045BD8952CE,0.607831
C6D9BA86-556B-E111-B43A-00505680000A,0.607831
315E317C-2AAB-E611-80E4-001DD8B72B61,0.573068
705EAEBC-CD60-ED11-9561-6045BD895B5A,0.573068
853F72D0-B6E3-EB11-8121-001DD8B72B61,0.573068
957A6EEC-E660-ED11-9561-6045BD895B5A,0.573068
9A225E32-7556-EC11-8F8F-000D3AAD44A7,0.573068


In [48]:
done_campaigns = df_campagne.loc[df_campagne.index == select_contact]['CampagneId']
done_campaigns

ContactPersoonId
C39094D7-60B1-EB11-811F-001DD8B72B62    D143743E-7B92-ED11-AAD1-6045BD895B5A
C39094D7-60B1-EB11-811F-001DD8B72B62    D143743E-7B92-ED11-AAD1-6045BD895B5A
C39094D7-60B1-EB11-811F-001DD8B72B62    D143743E-7B92-ED11-AAD1-6045BD895B5A
Name: CampagneId, dtype: object

In [49]:
similar_campagnes = df_campagne[df_campagne.index.isin(similar_users.index)].replace(0, np.nan).dropna(axis=1, how='all')
similar_campagnes

,CampagneId,CampagneNaam,Startdatum,rating
ContactPersoonId,,,,
01B45481-0877-E911-80FE-001DD8B72B62,D9834556-240C-EE11-8F6E-6045BD8956C9,OV-JO Community Event Starten NJ 2023,2023-11-29 16:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,A2F63005-8EA8-ED11-AAD1-6045BD895D85,OV-NW-Voka Bilan 2023,2023-05-25 16:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,C693DACA-04CF-ED11-B597-6045BD895805,OV-NW-Rode Loper Selectie-Gent Jazz-Gregory Po...,2023-07-05 19:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,317CD023-2B1E-ED11-B83D-000D3AAD783A,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,2022-12-21 19:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,6D357184-F5AF-EC11-9840-000D3A278F96,OV-P-GROEP H-SCALE UP-BRYO 2023,2023-05-30 09:00:00,10
...,...,...,...,...
E80EDB51-C66A-E111-B43A-00505680000A,317CD023-2B1E-ED11-B83D-000D3AAD783A,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,2022-12-21 19:00:00,10
E80EDB51-C66A-E111-B43A-00505680000A,A2F63005-8EA8-ED11-AAD1-6045BD895D85,OV-NW-Voka Bilan 2023,2023-05-25 16:00:00,10
E80EDB51-C66A-E111-B43A-00505680000A,317CD023-2B1E-ED11-B83D-000D3AAD783A,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,2022-12-21 19:00:00,10


In [50]:
#remove double campagneIds in similar_campagnes
similar_campagnes = similar_campagnes.drop_duplicates(subset=['CampagneId'])
similar_campagnes

,CampagneId,CampagneNaam,Startdatum,rating
ContactPersoonId,,,,
01B45481-0877-E911-80FE-001DD8B72B62,D9834556-240C-EE11-8F6E-6045BD8956C9,OV-JO Community Event Starten NJ 2023,2023-11-29 16:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,A2F63005-8EA8-ED11-AAD1-6045BD895D85,OV-NW-Voka Bilan 2023,2023-05-25 16:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,C693DACA-04CF-ED11-B597-6045BD895805,OV-NW-Rode Loper Selectie-Gent Jazz-Gregory Po...,2023-07-05 19:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,317CD023-2B1E-ED11-B83D-000D3AAD783A,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,2022-12-21 19:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,6D357184-F5AF-EC11-9840-000D3A278F96,OV-P-GROEP H-SCALE UP-BRYO 2023,2023-05-30 09:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,E1B0A3B0-8A4F-ED11-BBA2-6045BD895BFB,OV-NW-Voka|VeGHO Portlunch met Koen Schoors,2022-12-15 11:30:00,10
315E317C-2AAB-E611-80E4-001DD8B72B61,51FFF7D3-B591-ED11-AAD1-6045BD895D85,OV-Uitreiking VCDO - Cyclus 2022,2023-06-14 16:30:00,10
DC3AE779-BFBA-E511-9699-005056B06EC4,870BA609-A856-ED11-BBA2-6045BD895D85,OV-NW/A&O-XpertFinder HR Tech - 21 maart 2023,2023-03-21 08:00:00,10
DC3AE779-BFBA-E511-9699-005056B06EC4,B3865100-DEEF-ED11-8849-6045BD895233,OV-NW-Voka Netwekkers bij Well Played,2023-11-22 08:00:00,10


In [51]:
#only show campagnes that will start in the future
similar_campagnes = similar_campagnes[similar_campagnes['Startdatum'] > '2023-11-06']
similar_campagnes

,CampagneId,CampagneNaam,Startdatum,rating
ContactPersoonId,,,,
01B45481-0877-E911-80FE-001DD8B72B62,D9834556-240C-EE11-8F6E-6045BD8956C9,OV-JO Community Event Starten NJ 2023,2023-11-29 16:00:00,10
DC3AE779-BFBA-E511-9699-005056B06EC4,B3865100-DEEF-ED11-8849-6045BD895233,OV-NW-Voka Netwekkers bij Well Played,2023-11-22 08:00:00,10
